<a href="https://colab.research.google.com/github/alonba/eeg-bss/blob/main/FastICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne
!pip install pymatreader

In [1]:
import numpy as np
import mne as mne

In [28]:
# Fetch the data from the .set file
eeglab_epochs = mne.io.read_epochs_eeglab('./35_electrodes_filtered_epoched.set')

# Create the train set and test set
total_num_of_epochs = len(eeglab_epochs)

train_set_indexes = []
test_set_indexes = []
for i in range(total_num_of_epochs):
    random_num = np.random.rand()
    if (random_num < 0.8):  # 80% chance of being here
        train_set_indexes.append(i)
    else:
        test_set_indexes.append(i)

train_set = eeglab_epochs[train_set_indexes]
test_set = eeglab_epochs[test_set_indexes]

print(len(train_set))
print(len(test_set))

Extracting parameters from c:\Users\alonb\OneDrive - Technion\טכניון\סמסטר ט' - אביב 2022\פרוייקט א\eeg-bss\35_electrodes_filtered_epoched.set...
Not setting metadata
474 matching events found
No baseline correction applied
0 projection items activated
Ready.
380
94


In [30]:
# Export the Data
mne.export.export_epochs('train_set.set',train_set,overwrite=True)
mne.export.export_epochs('test_set.set',test_set,overwrite=True)

Overwriting existing file.
Overwriting existing file.


In [4]:
train_set = mne.io.read_epochs_eeglab('train_set.set')
train_ica = mne.preprocessing.ICA(n_components=35)  # Defaults to FastICA
train_ica.fit(train_set)

NameError: name 'mne' is not defined

In [3]:
train_set_left=train_set['Left']
train_set_right=train_set['Right']
train_set_no_move=train_set['NoMove']

NameError: name 'train_set' is not defined

In [2]:
train_left_reconstructed = []
train_right_reconstructed = []
train_no_move_reconstructed = []
for i in range(35):
    tmp_train_left = train_set_left.copy()
    tmp_train_right = train_set_right.copy()
    tmp_train_no_move = train_set_no_move.copy()
    train_left_reconstructed.append(train_ica.apply(tmp_train_left,include=i))
    train_right_reconstructed.append(train_ica.apply(tmp_train_right,include=i))
    train_no_move_reconstructed.append(train_ica.apply(tmp_train_no_move,include=i))


NameError: name 'train_set_left' is not defined